# Investigación para propuesta de valor de aplicación de libros

Durante la pandemia los libros recuperaron fortaleza como uno de los hobbies más populares tras presentarse la necesidad de confinamiento. Por esta razón surgieron muchas start-ups relacionadas con los libros y es importante conocer el mercado para poder realizar una propuesta de valor para una nueva aplicación de este ramo.

Por esta razón, a continuación se investigarán algunos valores relevantes para la propuesta de valor que se pretende realizar. Se compartió con nosotros una base de datos de uno de los competidores donde podremos encontrar información sobre libros, editoriales, autores, calificaciones de clientes y reseñas.

A continuación se enlistan los distintos apartados de la investigación:

* 1 - Conexión con la base de datos
* 2 - Estudio de las tablas
* 3 - Libros publicados después del 1 de enero de 2000
* 4 - Número de reseñas de usuarios y calificación promedio para cada libro
* 5 - Editorial que ha publicado el mayor número de libros 
* 6 - Autor que tiene la más alta calificación promedio por un libro
* 7 - Número promedio de reseñas de texto entre los usuarios
* 8 - Conclusiones

## Conexión con la base de datos

Para esta investigación se utilizará principalmente SQL y Python sólo servirá para guardar valores de los resultados, por lo que sólo son necesarias la librería de pandas y SQLalchemy. Procederemos a realizar estas importaciones y la conexión con la base de datos.

In [1]:
# Importar librerías
import pandas as pd
from sqlalchemy import create_engine
!pip install psycopg2-binary

# Conexión con base de datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Estudio de las tablas

A continuación para una mejor comprensión de las tablas dentro de la base de datos, se extraerán las 5 primeras lineas de cada una, así como su número de filas.

In [2]:
#Función para generar tablas

def primeras_5_lineas(tabla):
    print(tabla)
    display(pd.io.sql.read_sql('SELECT * FROM ' + tabla + ' LIMIT 5', con = engine))
    print(pd.io.sql.read_sql('SELECT COUNT(*) FROM ' + tabla, con = engine))
    print()
    print()

#Definición de tablas

tablas=['books', 'authors', 'publishers', 'ratings', 'reviews']

#Correr la función

for table in tablas:
    primeras_5_lineas(table)

books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


   count
0   1000


authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


   count
0    636


publishers


...

   count
0    340


ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


   count
0   6456


reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


   count
0   2793




Esta muestra de las primeras 5 filas de cada tabla nos ayuda a comprender mejor la estructura de las mismas y la estructura entre ellas, encontrando varias relaciones de índices.

# Libros publicados después del 1 de enero de 2000

En este caso será necesario filtrar la tabla 'books' por la columna de fechas de publicación para sólo obtener los libros del periodo determinado, y contarlos.

In [3]:
#Obtención del resultado

books_after_20000101 = pd.io.sql.read_sql("SELECT COUNT(book_id) AS book_amount FROM books WHERE publication_date >= '20000101'", con = engine)

#Impresión del resultado

print('Libros publicados después del 1 de enero de 2000:')
print(books_after_20000101)

Libros publicados después del 1 de enero de 2000:
   book_amount
0          821


Es interesante que la oferta de este competidor tiene la mayoría de sus libros muy recientes (82.1%), lo que indica que existe poca presencia de libros clásicos. Aquí podría existir una oferta de valor para la nueva aplicación.

## Número de reseñas de usuarios y calificación promedio para cada libro

En este caso será necesario utilizar tanto la tabla ratings como la tabla reviews y conectarlas con la tabla books a través del 'book_id', y a partir de esto obtener el conteo de la cantidad de reseñas y el promedio de la calificación.

In [4]:
#Obtención del resultado

tabla_reseñas_califmean = pd.io.sql.read_sql("""SELECT
    books.book_id as book_id,
    books.title as title,
    COUNT (reviews.review_id) as reviews,
    AVG (ratings.rating) as mean_rating
FROM
    books
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    books.book_id
ORDER BY
    books.book_id""", con = engine)
    
#Impresión del resultado

print('Reseñas y calificación promedio de libros:')
display(tabla_reseñas_califmean)   

Reseñas y calificación promedio de libros:


,book_id,title,reviews,mean_rating
0,1,'Salem's Lot,6,3.666667
1,2,1 000 Places to See Before You Die,2,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,9,4.666667
3,4,1491: New Revelations of the Americas Before C...,4,4.500000
4,5,1776,24,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),9,3.666667
996,997,Xenocide (Ender's Saga #3),15,3.400000
997,998,Year of Wonders,20,3.200000
998,999,You Suck (A Love Story #2),4,4.500000


Con esta tabla se podrán estudiar los libros más pouplares, así como la calificación con la que cuentan, dándonos una idea sobre qué libros puede ser valioso adquirir y en cuáles no tiene mucho sentido invertir.

## Editorial que ha publicado el mayor número de libros

En este caso, será necesario unir la tabla de editorial con la de libros a través del publisher_id, filtrar por aquellas publicaciones de más de 50 hojas, para evitar la inclusión de folletos y publicaciones similares, contar las publicaciones de cada editorial y mostrar la que tiene la mayoría.

In [5]:
#Obtención del resultado

editoriales_prinripales = pd.io.sql.read_sql("""SELECT
    publishers.publisher_id as publisher_id,
    publishers.publisher as publisher,
    COUNT (books.title) as books
FROM
    books
    LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
GROUP BY
    publishers.publisher_id
ORDER BY
    COUNT (books.title) DESC
LIMIT 5""", con = engine)

editorial_principal = pd.io.sql.read_sql("""SELECT
    publishers.publisher as publisher
FROM
    books
    LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
GROUP BY
    publishers.publisher_id
ORDER BY
    COUNT (books.title) DESC
LIMIT 1""", con = engine)
    
#Impresión del resultado

print('Principales 5 editoriales y número de libros:')
display(editoriales_prinripales)
print()
print()
print('Editorial con el mayor número de libros:')
print(editorial_principal)  

Principales 5 editoriales y número de libros:


,publisher_id,publisher,books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19




Editorial con el mayor número de libros:
       publisher
0  Penguin Books


En este caso, primero se imprimieron las 5 editoriales con mayor cantidad de libros con el propósito de comprobar que el primer lugar no fuera compartido, lo cual resultó ser cierto. La editorial con mayor publicación de libros es Penguin Book con 42 de ellos. Esta será una editorial muy importante para acordar un trato relacionado con la nueva aplicación.

## Autor que tiene la más alta calificación promedio por un libro

Para esta investigación será necesario unir la tabla de autores con la de libros y la de calificaciones y obtener la cuenta y promedio de calificaciones por libro. Filtrando por los libros con más de 50 calificaciones, con el propósito de obtener promedios relevantes, se obtendrá la calificación promedio y se ordenará por la misma, para definir el libro con la mayor calificación promedio y a partir de este resultado a su autor.

En este caso, igualmente se obtendrá previamente una tabla con los 5 libros mejor calificados para detectar si existe un empate.

In [6]:
#Obtención del resultado

libros_prinripales = pd.io.sql.read_sql("""SELECT
    autor,
    libro,
    mean_rating
    calificaciones
FROM
    (SELECT
    authors.author as autor,
    books.title as libro,
    AVG (ratings.rating) as mean_rating,
    COUNT(ratings.rating) as calificaciones
FROM
    books
    LEFT JOIN authors ON books.author_id = authors.author_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    books.book_id,
    authors.author
ORDER BY
    AVG (ratings.rating) DESC) AS subq
WHERE    
    calificaciones > 50 
LIMIT 5""", con = engine)

autor_principal = pd.io.sql.read_sql("""SELECT
    autor
FROM
    (SELECT
    authors.author as autor,
    books.title as libro,
    AVG (ratings.rating) as mean_rating,
    COUNT(ratings.rating) as calificaciones
FROM
    books
    LEFT JOIN authors ON books.author_id = authors.author_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    books.book_id,
    authors.author
ORDER BY
    AVG (ratings.rating) DESC) AS subq
WHERE    
    calificaciones > 50 
LIMIT 1""", con = engine)
    
#Impresión del resultado

print('Principales 5 libros por calificación promedio:')
display(libros_prinripales)
print()
print()
print('Autor con el libro mejor calificado:')
print(autor_principal)  

Principales 5 libros por calificación promedio:


,autor,libro,calificaciones
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634
1,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,4.391892
2,J.K. Rowling/Mary GrandPré,Harry Potter and the Chamber of Secrets (Harry...,4.287500
3,Markus Zusak/Cao Xuân Việt Khương,The Book Thief,4.264151
4,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,4.246575




Autor con el libro mejor calificado:
                        autor
0  J.K. Rowling/Mary GrandPré


Nuevamente no existió un empate, per la lista con los 5 libros mejor calificados nos ayudó a encontrar que tres de estos son precisamente del autor con la mejor calificación por un libro. Será importante estar al tanto de las publicaciones de este autor para adquirir los derechos de sus libros en cuanto estos salgan al mercado.

## Número promedio de reseñas de texto entre los usuarios con más de 50 calificaciones

En este caso se necesitarán la tabla de reseñas y la tabla de calificaiones. Primero a partir de la tabla de reseñas se creará una tabla que cuente la cantidad de reseñas que ha escrito cada usuario. Posteriormente se creará a partir de la tabla de calificaciones una tabla que cuente la cantidad de libros únicos calificados por los usuarios. Finalmente se unirán estas dos tablas, y filtrando por aquellos usuarios con más de 50 calificaciones, se obtendrá el promedio de reseñas.

In [7]:
#Obtención del resultado

promedio_reseñas = pd.io.sql.read_sql("""SELECT
    AVG(subreview.review_count)
FROM
    (SELECT
    username,
    COUNT(review_id) as review_count
FROM
    reviews
GROUP BY
    username) AS subreview
    LEFT JOIN (SELECT
    username,
    COUNT(DISTINCT book_id) as rating_count
FROM
    ratings
GROUP BY
    username) AS subrating ON subreview.username = subrating.username
WHERE    
    subrating.rating_count > 50 
LIMIT 1""", con = engine)

    
#Impresión del resultado

print('Promedio de reseñas por usuario:')
print(promedio_reseñas)

Promedio de reseñas por usuario:
         avg
0  24.333333


Con esto nos damos cuenta que los usuarios frecuentes están reseñando la mitad de los libros que califican, es una comunidad realmente activa, va a ser importante fomentar un ambiente donde los usuarios se sientan escuchados.

## Conclusiones

Los resultados obtenidos a partir de las búsquedas arrojaron los siguientes resultados:

* Libros en la base publicados a partir del año 2000: **821**
* Principal editorial: **Penguin books**
* Autor mejor calificado: **J.K. Rowling/Mary GrandPré**
* Promedio de reseñas de clientes que calificaron más de 50 libros: **24.33**

Aparte de esto, se obtuvo la tabla con los libros, su cantidad de reseñas y su promedio de calificación.

Esta información nos termina proveyendo de ciertas recomendaciones para la propuesta de valor de la nueva aplicación:

* Buscar un catálogo de libros más ámplio que incluya también obras clásicas.
* Buscar un acuerdo lo antes posible con la principal editorial.
* Dar seguimiento al autor mejor calificado para presentar sus libros en cuanto sean lanzados, especialmente ya que vimos que son 3 los libros de este autor los que están entre los 5 mejor calificados.
* Fomentar la retroalimentación entre la comunidad, ya que ésta está muy interesada en compartir sus puntos de vista.
* Buscar contratos para los libros más populares y mejor calificados, y aunque no evitar, no dar prioridad a libros que casi no generan ningún tipo de reacción.